<a href="https://colab.research.google.com/github/AfroGrit/jenga-stock/blob/wk5/wk5/_forex_01_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install yfinance
# !pip install plotly

In [2]:
# !pip install altair_viewer

In [3]:
!nvidia-smi  # this should display information about available GPUs


Mon Aug 12 10:05:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
import cudf  # this should work without any errors

In [5]:
# Numerical and Data Handling
import numpy as np
import pandas as pd

# Financial Data Sources
import yfinance as yf
import pandas_datareader as pdr

# Data Visualization
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt
import altair as alt
# import altair_viewer
import seaborn as sns

# Utilities
import time
from datetime import date
from tqdm import tqdm


## 0.0 helper functions

## 0.0 helper functions

In [6]:
def convert_to_utc(df, column_name='Date'):
    df[column_name] = pd.to_datetime(df[column_name], utc=True)
    return df

# Function to compute growth and extract growth columns
def get_growth(df: pd.DataFrame, prefix: str) -> pd.DataFrame:
    for i in [1, 3, 7, 30, 90, 365]:
        df[f'growth_{prefix}_{i}d'] = df['Adj Close'] / df['Adj Close'].shift(i)
    growth_keys = [k for k in df.keys() if k.startswith('growth')]
    return df[growth_keys].dropna()

# Function to download and process data for a list of tickers
def download_and_process_data(tickers: list) -> dict:
    return {
        ticker: get_growth(yf.download(ticker, period="max", interval="1d"), ticker)
        for ticker in tickers
    }

# Function to access and assign the growth data for each ticker to variables
def assign_growth_data_to_variables(data_dict: dict) -> dict:
    variables = {}
    for ticker in data_dict.keys():
        var_name = ticker.replace("-", "").replace("=", "").lower() + "_growth"
        variables[var_name] = data_dict[ticker]
    return variables

# Function to download and prepare financial data
def get_financials_df(ticker: str) -> pd.DataFrame:
    stock = yf.Ticker(ticker)
    # Download financial data
    financials = stock.financials.T
    return financials

# Function to generate dataframes for each stock's financial data
def generate_financial_dfs(tickers: dict) -> dict:
    financial_dfs = {}
    for name, ticker in tickers.items():
        df_name = f"{name.replace(' ', '_').lower()}_financials"
        financial_dfs[df_name] = get_financials_df(ticker)
    return financial_dfs

In [7]:
def ensure_datetime(df, date_column='Date'):
    """
    Ensure the date_column is in datetime format with no timezone.

    Parameters:
    df (pd.DataFrame): The DataFrame to process.
    date_column (str): The name of the date column to process.

    Returns:
    pd.DataFrame: The processed DataFrame with datetime formatted column.
    """
    if date_column in df.columns:
        df[date_column] = pd.to_datetime(df[date_column]).dt.tz_localize(None)
    else:
        df.index = pd.to_datetime(df.index).tz_localize(None)
        df.reset_index(inplace=True)
        df.rename(columns={'index': date_column}, inplace=True)
    return df



def remove_duplicate_columns(df) -> pd.DataFrame:
    """
    Remove duplicate columns in a DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame to process.

    Returns:
    pd.DataFrame: The processed DataFrame with duplicate columns removed.
    """
    return df.loc[:, ~df.columns.duplicated()]



def merge_dataframes(dataframes_dict, on='Date', how='left') -> pd.DataFrame:
    """
    Merge multiple DataFrames on a specified column.

    Parameters:
    dataframes_dict (dict): Dictionary of DataFrames to merge with keys as identifiers.
    on (str): The column name to merge on.
    how (str): The type of merge to be performed (default is 'left').

    Returns:
    pd.DataFrame: The merged DataFrame.
    """
    keys = list(dataframes_dict.keys())
    merged_df = ensure_datetime(dataframes_dict[keys[0]])

    for key in keys[1:]:
        df = ensure_datetime(dataframes_dict[key])
        merged_df = pd.merge(merged_df, df, on=on, how=how, suffixes=('', '_dup'))
        merged_df = remove_duplicate_columns(merged_df)

    return merged_df

In [46]:


# Function to download historical stock data
def download_stock_data(ticker, period="max", interval="1d"):
    return yf.download(tickers=ticker, period=period, interval=interval)

# Function to generate features for historical prices
def generate_historical_features(df, ticker):
    df['Ticker'] = ticker
    df['Year'] = df.index.year
    df['Month'] = df.index.month
    df['Weekday'] = df.index.weekday
    df['Date'] = df.index.date
    for i in [1, 3, 7, 30, 90, 365]:
        df[f'growth_{i}d'] = df['Adj Close'] / df['Adj Close'].shift(i)
    df['growth_future_5d'] = df['Adj Close'].shift(-5) / df['Adj Close']
    return df

# Function to add technical indicators
def add_technical_indicators(df):
    df['SMA10'] = df['Close'].rolling(10).mean()
    df['SMA20'] = df['Close'].rolling(20).mean()
    df['growing_moving_average'] = np.where(df['SMA10'] > df['SMA20'], 1, 0)
    df['high_minus_low_relative'] = (df.High - df.Low) / df['Adj Close']
    df['volatility'] = df['Adj Close'].rolling(30).std() * np.sqrt(252)
    df['is_positive_growth_5d_future'] = np.where(df['growth_future_5d'] > 1, 1, 0)
    return df

# Function to process all tickers
def process_all_tickers(tickers):
    stocks_df = pd.DataFrame({'A': []})
    for i, ticker in enumerate(tickers):
        print(i, ticker)

        # Download and process stock data
        historyPrices = download_stock_data(ticker)
        historyPrices = generate_historical_features(historyPrices, ticker)
        historyPrices = add_technical_indicators(historyPrices)

        # Sleep 1 second between downloads
        time.sleep(1)

        # Concatenate dataframes
        if stocks_df.empty:
            stocks_df = historyPrices
        else:
            stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

    return stocks_df

# def get_ticker_type(ticker:str, african_currency_pairs_daily, forex_currency_pairs_daily):
#   if ticker in african_currency_pairs_daily:
#     return 'africa'
#   elif ticker in forex_currency_pairs_daily:
#     return 'other'
#   else:
#     return 'err'


def get_ticker_type(
    ticker:str,
    african_currency_pairs_daily,
    usa_currency_pairs_daily,
    south_america_currency_pairs_daily,
    europe_currency_pairs_daily,
    asia_currency_pairs_daily
    ):
  if ticker in african_currency_pairs_daily:
    return 'africa'
  elif ticker in usa_currency_pairs_daily:
    return 'usa'
  elif ticker in south_america_currency_pairs_daily:
    return 'south_america'
  elif ticker in europe_currency_pairs_daily:
    return 'europe'
  elif ticker in asia_currency_pairs_daily:
    return 'asia'
  else:
    return 'err'

def write_dataframe_to_parquet(df, filepath):
    """
    Write a DataFrame to a Parquet file.

    Parameters:
    df (pd.DataFrame): The DataFrame to be written to disk.
    filepath (str): The file path where the Parquet file will be saved.
    """
    df.to_parquet(filepath, engine='pyarrow')

## 1.1. Financial Ratio

Uncertain if we need this

In [11]:
# def convert_to_utc(df, column_name='Date'):
#     df[column_name] = pd.to_datetime(df[column_name], utc=True)
#     return df

# # Function to compute growth and extract growth columns
# def get_growth(df: pd.DataFrame, prefix: str) -> pd.DataFrame:
#     for i in [1, 3, 7, 30, 90, 365]:
#         df[f'growth_{prefix}_{i}d'] = df['Adj Close'] / df['Adj Close'].shift(i)
#     growth_keys = [k for k in df.keys() if k.startswith('growth')]
#     return df[growth_keys].dropna()

# # Function to download and process data for a list of tickers
# def download_and_process_data(tickers: list) -> dict:
#     return {
#         ticker: get_growth(yf.download(ticker, period="max", interval="1d"), ticker)
#         for ticker in tickers
#     }

# # Function to access and assign the growth data for each ticker to variables
# def assign_growth_data_to_variables(data_dict: dict) -> dict:
#     variables = {}
#     for ticker in data_dict.keys():
#         var_name = ticker.replace("-", "").replace("=", "").lower() + "_growth"
#         variables[var_name] = data_dict[ticker]
#     return variables

# # Function to download and prepare financial data
# def get_financials_df(ticker: str) -> pd.DataFrame:
#     stock = yf.Ticker(ticker)
#     # Download financial data
#     financials = stock.financials.T
#     return financials

# # Function to generate dataframes for each stock's financial data
# def generate_financial_dfs(tickers: dict) -> dict:
#     financial_dfs = {}
#     for name, ticker in tickers.items():
#         df_name = f"{name.replace(' ', '_').lower()}_financials"
#         financial_dfs[df_name] = get_financials_df(ticker)
#     return financial_dfs

In [12]:
# def ensure_datetime(df, date_column='Date'):
#     """
#     Ensure the date_column is in datetime format with no timezone.

#     Parameters:
#     df (pd.DataFrame): The DataFrame to process.
#     date_column (str): The name of the date column to process.

#     Returns:
#     pd.DataFrame: The processed DataFrame with datetime formatted column.
#     """
#     if date_column in df.columns:
#         df[date_column] = pd.to_datetime(df[date_column]).dt.tz_localize(None)
#     else:
#         df.index = pd.to_datetime(df.index).tz_localize(None)
#         df.reset_index(inplace=True)
#         df.rename(columns={'index': date_column}, inplace=True)
#     return df



# def remove_duplicate_columns(df) -> pd.DataFrame:
#     """
#     Remove duplicate columns in a DataFrame.

#     Parameters:
#     df (pd.DataFrame): The DataFrame to process.

#     Returns:
#     pd.DataFrame: The processed DataFrame with duplicate columns removed.
#     """
#     return df.loc[:, ~df.columns.duplicated()]



# def merge_dataframes(dataframes_dict, on='Date', how='left') -> pd.DataFrame:
#     """
#     Merge multiple DataFrames on a specified column.

#     Parameters:
#     dataframes_dict (dict): Dictionary of DataFrames to merge with keys as identifiers.
#     on (str): The column name to merge on.
#     how (str): The type of merge to be performed (default is 'left').

#     Returns:
#     pd.DataFrame: The merged DataFrame.
#     """
#     keys = list(dataframes_dict.keys())
#     merged_df = ensure_datetime(dataframes_dict[keys[0]])

#     for key in keys[1:]:
#         df = ensure_datetime(dataframes_dict[key])
#         merged_df = pd.merge(merged_df, df, on=on, how=how, suffixes=('', '_dup'))
#         merged_df = remove_duplicate_columns(merged_df)

#     return merged_df

In [13]:
# def convert_to_utc(df, column_name='Date'):
#     df[column_name] = pd.to_datetime(df[column_name], utc=True)
#     return df

# # Function to compute growth and extract growth columns
# def get_growth(df: pd.DataFrame, prefix: str) -> pd.DataFrame:
#     for i in [1, 3, 7, 30, 90, 365]:
#         df[f'growth_{prefix}_{i}d'] = df['Adj Close'] / df['Adj Close'].shift(i)
#     growth_keys = [k for k in df.keys() if k.startswith('growth')]
#     return df[growth_keys].dropna()

# # Function to download and process data for a list of tickers
# def download_and_process_data(tickers: list) -> dict:
#     return {
#         ticker: get_growth(yf.download(ticker, period="max", interval="1d"), ticker)
#         for ticker in tickers
#     }

# # Function to access and assign the growth data for each ticker to variables
# def assign_growth_data_to_variables(data_dict: dict) -> dict:
#     variables = {}
#     for ticker in data_dict.keys():
#         var_name = ticker.replace("-", "").replace("=", "").lower() + "_growth"
#         variables[var_name] = data_dict[ticker]
#     return variables

# # Function to download and prepare financial data
# def get_financials_df(ticker: str) -> pd.DataFrame:
#     stock = yf.Ticker(ticker)
#     # Download financial data
#     financials = stock.financials.T
#     return financials

# # Function to generate dataframes for each stock's financial data
# def generate_financial_dfs(tickers: dict) -> dict:
#     financial_dfs = {}
#     for name, ticker in tickers.items():
#         df_name = f"{name.replace(' ', '_').lower()}_financials"
#         financial_dfs[df_name] = get_financials_df(ticker)
#     return financial_dfs

In [14]:
# def ensure_datetime(df):
#     """
#     Ensure the 'Date' column is in datetime format without timezone.

#     Parameters:
#     df (pd.DataFrame): DataFrame to process.

#     Returns:
#     pd.DataFrame: DataFrame with 'Date' column in datetime format.
#     """
#     df['Date'] = pd.to_datetime(df['Date']).dt.tz_localize(None)
#     return df

# def remove_duplicate_columns(df):
#     """
#     Remove duplicate columns resulting from merge operations.

#     Parameters:
#     df (pd.DataFrame): DataFrame to process.

#     Returns:
#     pd.DataFrame: DataFrame with duplicate columns removed.
#     """
#     cols_to_drop = [col for col in df.columns if '_dup' in col]
#     df = df.drop(columns=cols_to_drop)
#     return df

# def final_merge_dataframes(dataframe_names, on='Date', how='left') -> pd.DataFrame:
#     """
#     Merge multiple DataFrames on a specified column.

#     Parameters:
#     dataframe_names (list of str): List of DataFrame names to merge.
#     on (str): The column name to merge on.
#     how (str): The type of merge to be performed (default is 'left').

#     Returns:
#     pd.DataFrame: The merged DataFrame.
#     """
#     if not dataframe_names:
#         raise ValueError("The list of DataFrame names is empty.")

#     # Initialize with the first DataFrame
#     merged_df = ensure_datetime(globals()[dataframe_names[0]])

#     for name in dataframe_names[1:]:
#         df = ensure_datetime(globals()[name])
#         merged_df = pd.merge(merged_df, df, on=on, how=how, suffixes=('', '_dup'))
#         merged_df = remove_duplicate_columns(merged_df)

#     return merged_df



# data

## 1.0. Financial Ratio

## 2.0. Technical Indicators with TA-lib

In [15]:
# https://stackoverflow.com/questions/49648391/how-to-install-ta-lib-in-google-colab
# Update (apr 2023): Colab is now Python 3.10
# there are some other (older) ways to install talib

url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3
import talib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4083    0  4083    0     0  15602      0 --:--:-- --:--:-- --:--:-- 15583
100  517k  100  517k    0     0   485k      0  0:00:01  0:00:01 --:--:--  485k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4115    0  4115    0     0  17753      0 --:--:-- --:--:-- --:--:-- 17813
100  392k  100  392k    0     0   399k      0 --:--:-- --:--:-- --:--:-- 4576k


### 2.1. Momentum indicators

In [16]:
def talib_get_momentum_indicators_for_one_ticker(df: pd.DataFrame) -> pd.DataFrame:
  # ADX - Average Directional Movement Index
  talib_momentum_adx = talib.ADX(df.High.values, df.Low.values, df.Close.values, timeperiod=14)
  # ADXR - Average Directional Movement Index Rating
  talib_momentum_adxr = talib.ADXR(df.High.values, df.Low.values, df.Close.values, timeperiod=14 )
  # APO - Absolute Price Oscillator
  talib_momentum_apo = talib.APO(df.Close.values, fastperiod=12, slowperiod=26, matype=0 )
  # AROON - Aroon
  talib_momentum_aroon = talib.AROON(df.High.values, df.Low.values, timeperiod=14 )
  # talib_momentum_aroon[0].size
  # talib_momentum_aroon[1].size
  # AROONOSC - Aroon Oscillator
  talib_momentum_aroonosc = talib.AROONOSC(df.High.values, df.Low.values, timeperiod=14)
  # BOP - Balance of Power
  # https://school.stockcharts.com/doku.php?id=technical_indicators:balance_of_power
     #calculate open prices as shifted closed prices from the prev day
     # open = df.Last.shift(1)
  talib_momentum_bop = talib.BOP(df.Open.values, df.High.values, df.Low.values, df.Close.values)
  # CCI - Commodity Channel Index
  talib_momentum_cci = talib.CCI(df.High.values, df.Low.values, df.Close.values, timeperiod=14)
  # CMO - Chande Momentum Oscillator
  talib_momentum_cmo = talib.CMO(df.Close.values, timeperiod=14)
  # DX - Directional Movement Index
  talib_momentum_dx = talib.DX(df.High.values, df.Low.values, df.Close.values, timeperiod=14)
  # MACD - Moving Average Convergence/Divergence
  talib_momentum_macd, talib_momentum_macdsignal, talib_momentum_macdhist = talib.MACD(df.Close.values, fastperiod=12, \
                                                                                       slowperiod=26, signalperiod=9)
  # MACDEXT - MACD with controllable MA type
  talib_momentum_macd_ext, talib_momentum_macdsignal_ext, talib_momentum_macdhist_ext = talib.MACDEXT(df.Close.values, \
                                                                                                    fastperiod=12, \
                                                                                                    fastmatype=0, \
                                                                                                    slowperiod=26, \
                                                                                                    slowmatype=0, \
                                                                                                    signalperiod=9, \
                                                                                                  signalmatype=0)
  # MACDFIX - Moving Average Convergence/Divergence Fix 12/26
  talib_momentum_macd_fix, talib_momentum_macdsignal_fix, talib_momentum_macdhist_fix = talib.MACDFIX(df.Close.values, \
                                                                                                      signalperiod=9)
  # MFI - Money Flow Index
  talib_momentum_mfi = talib.MFI(df.High.values, df.Low.values, df.Close.values, df.Volume.values, timeperiod=14)
  # MINUS_DI - Minus Directional Indicator
  talib_momentum_minus_di = talib.MINUS_DM(df.High.values, df.Low.values, timeperiod=14)
  # MOM - Momentum
  talib_momentum_mom = talib.MOM(df.Close.values, timeperiod=10)
  # PLUS_DI - Plus Directional Indicator
  talib_momentum_plus_di = talib.PLUS_DI(df.High.values, df.Low.values, df.Close.values, timeperiod=14)
  # PLUS_DM - Plus Directional Movement
  talib_momentum_plus_dm = talib.PLUS_DM(df.High.values, df.Low.values, timeperiod=14)
  # PPO - Percentage Price Oscillator
  talib_momentum_ppo = talib.PPO(df.Close.values, fastperiod=12, slowperiod=26, matype=0)
  # ROC - Rate of change : ((price/prevPrice)-1)*100
  talib_momentum_roc = talib.ROC(df.Close.values, timeperiod=10)
  # ROCP - Rate of change Percentage: (price-prevPrice)/prevPrice
  talib_momentum_rocp = talib.ROCP(df.Close.values, timeperiod=10)
  # ROCR - Rate of change ratio: (price/prevPrice)
  talib_momentum_rocr = talib.ROCR(df.Close.values, timeperiod=10)
  # ROCR100 - Rate of change ratio 100 scale: (price/prevPrice)*100
  talib_momentum_rocr100 = talib.ROCR100(df.Close.values, timeperiod=10)
  # RSI - Relative Strength Index
  talib_momentum_rsi = talib.RSI(df.Close.values, timeperiod=14)
  # STOCH - Stochastic
  talib_momentum_slowk, talib_momentum_slowd = talib.STOCH(df.High.values, df.Low.values, df.Close.values, \
                                                           fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
  # STOCHF - Stochastic Fast
  talib_momentum_fastk, talib_momentum_fastd = talib.STOCHF(df.High.values, df.Low.values, df.Close.values, \
                                                            fastk_period=5, fastd_period=3, fastd_matype=0)
  # STOCHRSI - Stochastic Relative Strength Index
  talib_momentum_fastk_rsi, talib_momentum_fastd_rsi = talib.STOCHRSI(df.Close.values, timeperiod=14, \
                                                                      fastk_period=5, fastd_period=3, fastd_matype=0)
  # TRIX - 1-day Rate-Of-Change (ROC) of a Triple Smooth EMA
  talib_momentum_trix = talib.TRIX(df.Close.values, timeperiod=30)
  # ULTOSC - Ultimate Oscillator
  talib_momentum_ultosc = talib.ULTOSC(df.High.values, df.Low.values, df.Close.values, timeperiod1=7, timeperiod2=14, timeperiod3=28)
  # WILLR - Williams' %R
  talib_momentum_willr = talib.WILLR(df.High.values, df.Low.values, df.Close.values, timeperiod=14)

  momentum_df =   pd.DataFrame(
    {
      # assume here multi-index
      # 'datetime': df.index.get_level_values(0),
      # 'ticker': df.index.get_level_values(1) ,

      # old way with separate columns
      'Date': df.Date.values,
      'Ticker': df.Ticker,

      'adx': talib_momentum_adx,
      'adxr': talib_momentum_adxr,
      'apo': talib_momentum_apo,
      'aroon_1': talib_momentum_aroon[0] ,
      'aroon_2': talib_momentum_aroon[1],
      'aroonosc': talib_momentum_aroonosc,
      'bop': talib_momentum_bop,
      'cci': talib_momentum_cci,
      'cmo': talib_momentum_cmo,
      'dx': talib_momentum_dx,
      'macd': talib_momentum_macd,
      'macdsignal': talib_momentum_macdsignal,
      'macdhist': talib_momentum_macdhist,
      'macd_ext': talib_momentum_macd_ext,
      'macdsignal_ext': talib_momentum_macdsignal_ext,
      'macdhist_ext': talib_momentum_macdhist_ext,
      'macd_fix': talib_momentum_macd_fix,
      'macdsignal_fix': talib_momentum_macdsignal_fix,
      'macdhist_fix': talib_momentum_macdhist_fix,
      'mfi': talib_momentum_mfi,
      'minus_di': talib_momentum_minus_di,
      'mom': talib_momentum_mom,
      'plus_di': talib_momentum_plus_di,
      'dm': talib_momentum_plus_dm,
      'ppo': talib_momentum_ppo,
      'roc': talib_momentum_roc,
      'rocp': talib_momentum_rocp,
      'rocr': talib_momentum_rocr,
      'rocr100': talib_momentum_rocr100,
      'rsi': talib_momentum_rsi,
      'slowk': talib_momentum_slowk,
      'slowd': talib_momentum_slowd,
      'fastk': talib_momentum_fastk,
      'fastd': talib_momentum_fastd,
      'fastk_rsi': talib_momentum_fastk_rsi,
      'fastd_rsi': talib_momentum_fastd_rsi,
      'trix': talib_momentum_trix,
      'ultosc': talib_momentum_ultosc,
      'willr': talib_momentum_willr,
     }
  )
  return momentum_df


### 2.2. Volume, Volatility, Cycle, Price indicators

In [17]:
def talib_get_volume_volatility_cycle_price_indicators(df: pd.DataFrame) -> pd.DataFrame:
        # TA-Lib Volume indicators
        # https://github.com/TA-Lib/ta-lib-python/blob/master/docs/func_groups/volume_indicators.md
        # AD - Chaikin A/D Line
        talib_ad = talib.AD(
            df.High.values, df.Low.values, df.Close.values, df.Volume.values)
        # ADOSC - Chaikin A/D Oscillator
        talib_adosc = talib.ADOSC(
            df.High.values, df.Low.values, df.Close.values, df.Volume.values, fastperiod=3, slowperiod=10)
        # OBV - On Balance Volume
        talib_obv = talib.OBV(
            df.Close.values, df.Volume.values)

        # TA-Lib Volatility indicators
        # https://github.com/TA-Lib/ta-lib-python/blob/master/docs/func_groups/volatility_indicators.md
        # ATR - Average True Range
        talib_atr = talib.ATR(
            df.High.values, df.Low.values, df.Close.values, timeperiod=14)
        # NATR - Normalized Average True Range
        talib_natr = talib.NATR(
            df.High.values, df.Low.values, df.Close.values, timeperiod=14)
        # OBV - On Balance Volume
        talib_obv = talib.OBV(
            df.Close.values, df.Volume.values)

        # TA-Lib Cycle Indicators
        # https://github.com/TA-Lib/ta-lib-python/blob/master/docs/func_groups/cycle_indicators.md
        # HT_DCPERIOD - Hilbert Transform - Dominant Cycle Period
        talib_ht_dcperiod = talib.HT_DCPERIOD(df.Close.values)
        # HT_DCPHASE - Hilbert Transform - Dominant Cycle Phase
        talib_ht_dcphase = talib.HT_DCPHASE(df.Close.values)
        # HT_PHASOR - Hilbert Transform - Phasor Components
        talib_ht_phasor_inphase, talib_ht_phasor_quadrature = talib.HT_PHASOR(
            df.Close.values)
        # HT_SINE - Hilbert Transform - SineWave
        talib_ht_sine_sine, talib_ht_sine_leadsine = talib.HT_SINE(
            df.Close.values)
        # HT_TRENDMODE - Hilbert Transform - Trend vs Cycle Mode
        talib_ht_trendmode = talib.HT_TRENDMODE(df.Close.values)

        # TA-Lib Price Transform Functions
        # https://github.com/TA-Lib/ta-lib-python/blob/master/docs/func_groups/price_transform.md
        # AVGPRICE - Average Price
        talib_avgprice = talib.AVGPRICE(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # MEDPRICE - Median Price
        talib_medprice = talib.MEDPRICE(df.High.values, df.Low.values)
        # TYPPRICE - Typical Price
        talib_typprice = talib.TYPPRICE(
            df.High.values, df.Low.values, df.Close.values)
        # WCLPRICE - Weighted Close Price
        talib_wclprice = talib.WCLPRICE(
            df.High.values, df.Low.values, df.Close.values)

        volume_volatility_cycle_price_df = pd.DataFrame(
            {'Date': df.Date.values,
             'Ticker': df.Ticker,
             # TA-Lib Volume indicators
             'ad': talib_ad,
             'adosc': talib_adosc,
             'obv': talib_obv,
             # TA-Lib Volatility indicators
             'atr': talib_atr,
             'natr': talib_natr,
             'obv': talib_obv,
             # TA-Lib Cycle Indicators
             'ht_dcperiod': talib_ht_dcperiod,
             'ht_dcphase': talib_ht_dcphase,
             'ht_phasor_inphase': talib_ht_phasor_inphase,
             'ht_phasor_quadrature': talib_ht_phasor_quadrature,
             'ht_sine_sine': talib_ht_sine_sine,
             'ht_sine_leadsine': talib_ht_sine_leadsine,
             'ht_trendmod': talib_ht_trendmode,
             # TA-Lib Price Transform Functions
             'avgprice': talib_avgprice,
             'medprice': talib_medprice,
             'typprice': talib_typprice,
             'wclprice': talib_wclprice,
             }
        )

        # Need a proper date type
        volume_volatility_cycle_price_df['Date'] = pd.to_datetime(
            volume_volatility_cycle_price_df['Date'])

        return volume_volatility_cycle_price_df


### 2.3. Pattern indicators

In [18]:
def talib_get_pattern_recognition_indicators(df: pd.DataFrame) -> pd.DataFrame:
# TA-Lib Pattern Recognition indicators
        # https://github.com/TA-Lib/ta-lib-python/blob/master/docs/func_groups/pattern_recognition.md
        # Nice article about candles (pattern recognition) https://medium.com/analytics-vidhya/recognizing-over-50-candlestick-patterns-with-python-4f02a1822cb5

        # CDL2CROWS - Two Crows
        talib_cdl2crows = talib.CDL2CROWS(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDL3BLACKCROWS - Three Black Crows
        talib_cdl3blackrows = talib.CDL3BLACKCROWS(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDL3INSIDE - Three Inside Up/Down
        talib_cdl3inside = talib.CDL3INSIDE(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDL3LINESTRIKE - Three-Line Strike
        talib_cdl3linestrike = talib.CDL3LINESTRIKE(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDL3OUTSIDE - Three Outside Up/Down
        talib_cdl3outside = talib.CDL3OUTSIDE(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDL3STARSINSOUTH - Three Stars In The South
        talib_cdl3starsinsouth = talib.CDL3STARSINSOUTH(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDL3WHITESOLDIERS - Three Advancing White Soldiers
        talib_cdl3whitesoldiers = talib.CDL3WHITESOLDIERS(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLABANDONEDBABY - Abandoned Baby
        talib_cdlabandonedbaby = talib.CDLABANDONEDBABY(
            df.Open.values, df.High.values, df.Low.values, df.Close.values, penetration=0)
        # CDLADVANCEBLOCK - Advance Block
        talib_cdladvancedblock = talib.CDLADVANCEBLOCK(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLBELTHOLD - Belt-hold
        talib_cdlbelthold = talib.CDLBELTHOLD(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLBREAKAWAY - Breakaway
        talib_cdlbreakaway = talib.CDLBREAKAWAY(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLCLOSINGMARUBOZU - Closing Marubozu
        talib_cdlclosingmarubozu = talib.CDLCLOSINGMARUBOZU(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLCONCEALBABYSWALL - Concealing Baby Swallow
        talib_cdlconcealbabyswall = talib.CDLCONCEALBABYSWALL(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLCOUNTERATTACK - Counterattack
        talib_cdlcounterattack = talib.CDLCOUNTERATTACK(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLDARKCLOUDCOVER - Dark Cloud Cover
        talib_cdldarkcloudcover = talib.CDLDARKCLOUDCOVER(
            df.Open.values, df.High.values, df.Low.values, df.Close.values, penetration=0)
        # CDLDOJI - Doji
        talib_cdldoji = talib.CDLDOJI(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLDOJISTAR - Doji Star
        talib_cdldojistar = talib.CDLDOJISTAR(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLDRAGONFLYDOJI - Dragonfly Doji
        talib_cdldragonflydoji = talib.CDLDRAGONFLYDOJI(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLENGULFING - Engulfing Pattern
        talib_cdlengulfing = talib.CDLENGULFING(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)

        # CDLEVENINGDOJISTAR - Evening Doji Star
        talib_cdleveningdojistar = talib.CDLEVENINGDOJISTAR(
            df.Open.values, df.High.values, df.Low.values, df.Close.values, penetration=0)
        # CDLEVENINGSTAR - Evening Star
        talib_cdleveningstar = talib.CDLEVENINGSTAR(
            df.Open.values, df.High.values, df.Low.values, df.Close.values, penetration=0)
        # CDLGAPSIDESIDEWHITE - Up/Down-gap side-by-side white lines
        talib_cdlgapsidesidewhite = talib.CDLGAPSIDESIDEWHITE(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLGRAVESTONEDOJI - Gravestone Doji
        talib_cdlgravestonedoji = talib.CDLGRAVESTONEDOJI(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLHAMMER - Hammer
        talib_cdlhammer = talib.CDLHAMMER(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLHANGINGMAN - Hanging Man
        talib_cdlhangingman = talib.CDLHANGINGMAN(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLHARAMI - Harami Pattern
        talib_cdlharami = talib.CDLHARAMI(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLHARAMICROSS - Harami Cross Pattern
        talib_cdlharamicross = talib.CDLHARAMICROSS(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLHIGHWAVE - High-Wave Candle
        talib_cdlhighwave = talib.CDLHIGHWAVE(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLHIKKAKE - Hikkake Pattern
        talib_cdlhikkake = talib.CDLHIKKAKE(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLHIKKAKEMOD - Modified Hikkake Pattern
        talib_cdlhikkakemod = talib.CDLHIKKAKEMOD(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)

        # CDLHOMINGPIGEON - Homing Pigeon
        talib_cdlhomingpigeon = talib.CDLHOMINGPIGEON(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLIDENTICAL3CROWS - Identical Three Crows
        talib_cdlidentical3crows = talib.CDLIDENTICAL3CROWS(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLINNECK - In-Neck Pattern
        talib_cdlinneck = talib.CDLINNECK(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLINVERTEDHAMMER - Inverted Hammer
        talib_cdlinvertedhammer = talib.CDLINVERTEDHAMMER(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLKICKING - Kicking
        talib_cdlkicking = talib.CDLKICKING(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLKICKINGBYLENGTH - Kicking - bull/bear determined by the longer marubozu
        talib_cdlkickingbylength = talib.CDLKICKINGBYLENGTH(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLLADDERBOTTOM - Ladder Bottom
        talib_cdlladderbottom = talib.CDLLADDERBOTTOM(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLLONGLEGGEDDOJI - Long Legged Doji
        talib_cdllongleggeddoji = talib.CDLLONGLEGGEDDOJI(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLLONGLINE - Long Line Candle
        talib_cdllongline = talib.CDLLONGLINE(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLMARUBOZU - Marubozu
        talib_cdlmarubozu = talib.CDLMARUBOZU(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLMATCHINGLOW - Matching Low
        talib_cdlmatchinglow = talib.CDLMATCHINGLOW(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)

        # CDLMATHOLD - Mat Hold
        talib_cdlmathold = talib.CDLMATHOLD(
            df.Open.values, df.High.values, df.Low.values, df.Close.values, penetration=0)
        # CDLMORNINGDOJISTAR - Morning Doji Star
        talib_cdlmorningdojistar = talib.CDLMORNINGDOJISTAR(
            df.Open.values, df.High.values, df.Low.values, df.Close.values, penetration=0)
        # CDLMORNINGSTAR - Morning Star
        talib_cdlmorningstar = talib.CDLMORNINGSTAR(
            df.Open.values, df.High.values, df.Low.values, df.Close.values, penetration=0)
        # CDLONNECK - On-Neck Pattern
        talib_cdlonneck = talib.CDLONNECK(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLPIERCING - Piercing Pattern
        talib_cdlpiercing = talib.CDLPIERCING(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLRICKSHAWMAN - Rickshaw Man
        talib_cdlrickshawman = talib.CDLRICKSHAWMAN(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLRISEFALL3METHODS - Rising/Falling Three Methods
        talib_cdlrisefall3methods = talib.CDLRISEFALL3METHODS(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLSEPARATINGLINES - Separating Lines
        talib_cdlseparatinglines = talib.CDLSEPARATINGLINES(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLSHOOTINGSTAR - Shooting Star
        talib_cdlshootingstar = talib.CDLSHOOTINGSTAR(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLSHORTLINE - Short Line Candle
        talib_cdlshortline = talib.CDLSHORTLINE(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLSPINNINGTOP - Spinning Top
        talib_cdlspinningtop = talib.CDLSPINNINGTOP(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)

        # CDLSTALLEDPATTERN - Stalled Pattern
        talib_cdlstalledpattern = talib.CDLSTALLEDPATTERN(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLSTICKSANDWICH - Stick Sandwich
        talib_cdlsticksandwich = talib.CDLSTICKSANDWICH(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLTAKURI - Takuri (Dragonfly Doji with very long lower shadow)
        talib_cdltakuru = talib.CDLTAKURI(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLTASUKIGAP - Tasuki Gap
        talib_cdltasukigap = talib.CDLTASUKIGAP(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLTHRUSTING - Thrusting Pattern
        talib_cdlthrusting = talib.CDLTHRUSTING(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLTRISTAR - Tristar Pattern
        talib_cdltristar = talib.CDLTRISTAR(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLUNIQUE3RIVER - Unique 3 River
        talib_cdlunique3river = talib.CDLUNIQUE3RIVER(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLUPSIDEGAP2CROWS - Upside Gap Two Crows
        talib_cdlupsidegap2crows = talib.CDLUPSIDEGAP2CROWS(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)
        # CDLXSIDEGAP3METHODS - Upside/Downside Gap Three Methods
        talib_cdlxsidegap3methods = talib.CDLXSIDEGAP3METHODS(
            df.Open.values, df.High.values, df.Low.values, df.Close.values)

        pattern_indicators_df = pd.DataFrame(
            {'Date': df.Date.values,
             'Ticker': df.Ticker,
             # TA-Lib Pattern Recognition indicators
             'cdl2crows': talib_cdl2crows,
             'cdl3blackrows': talib_cdl3blackrows,
             'cdl3inside': talib_cdl3inside,
             'cdl3linestrike': talib_cdl3linestrike,
             'cdl3outside': talib_cdl3outside,
             'cdl3starsinsouth': talib_cdl3starsinsouth,
             'cdl3whitesoldiers': talib_cdl3whitesoldiers,
             'cdlabandonedbaby': talib_cdlabandonedbaby,
             'cdladvancedblock': talib_cdladvancedblock,
             'cdlbelthold': talib_cdlbelthold,
             'cdlbreakaway': talib_cdlbreakaway,
             'cdlclosingmarubozu': talib_cdlclosingmarubozu,
             'cdlconcealbabyswall': talib_cdlconcealbabyswall,
             'cdlcounterattack': talib_cdlcounterattack,
             'cdldarkcloudcover': talib_cdldarkcloudcover,
             'cdldoji': talib_cdldoji,
             'cdldojistar': talib_cdldojistar,
             'cdldragonflydoji': talib_cdldragonflydoji,
             'cdlengulfing': talib_cdlengulfing,
             'cdleveningdojistar': talib_cdleveningdojistar,
             'cdleveningstar': talib_cdleveningstar,
             'cdlgapsidesidewhite': talib_cdlgapsidesidewhite,
             'cdlgravestonedoji': talib_cdlgravestonedoji,
             'cdlhammer': talib_cdlhammer,
             'cdlhangingman': talib_cdlhangingman,
             'cdlharami': talib_cdlharami,
             'cdlharamicross': talib_cdlharamicross,
             'cdlhighwave': talib_cdlhighwave,
             'cdlhikkake': talib_cdlhikkake,
             'cdlhikkakemod': talib_cdlhikkakemod,
             'cdlhomingpigeon': talib_cdlhomingpigeon,
             'cdlidentical3crows': talib_cdlidentical3crows,
             'cdlinneck': talib_cdlinneck,
             'cdlinvertedhammer': talib_cdlinvertedhammer,
             'cdlkicking': talib_cdlkicking,
             'cdlkickingbylength': talib_cdlkickingbylength,
             'cdlladderbottom': talib_cdlladderbottom,
             'cdllongleggeddoji': talib_cdllongleggeddoji,
             'cdllongline': talib_cdllongline,
             'cdlmarubozu': talib_cdlmarubozu,
             'cdlmatchinglow': talib_cdlmatchinglow,
             'cdlmathold': talib_cdlmathold,
             'cdlmorningdojistar': talib_cdlmorningdojistar,
             'cdlmorningstar': talib_cdlmorningstar,
             'cdlonneck': talib_cdlonneck,
             'cdlpiercing': talib_cdlpiercing,
             'cdlrickshawman': talib_cdlrickshawman,
             'cdlrisefall3methods': talib_cdlrisefall3methods,
             'cdlseparatinglines': talib_cdlseparatinglines,
             'cdlshootingstar': talib_cdlshootingstar,
             'cdlshortline': talib_cdlshortline,
             'cdlspinningtop': talib_cdlspinningtop,
             'cdlstalledpattern': talib_cdlstalledpattern,
             'cdlsticksandwich': talib_cdlsticksandwich,
             'cdltakuru': talib_cdltakuru,
             'cdltasukigap': talib_cdltasukigap,
             'cdlthrusting': talib_cdlthrusting,
             'cdltristar': talib_cdltristar,
             'cdlunique3river': talib_cdlunique3river,
             'cdlupsidegap2crows': talib_cdlupsidegap2crows,
             'cdlxsidegap3methods': talib_cdlxsidegap3methods
             }
        )

        # Need a proper date type
        pattern_indicators_df['Date'] = pd.to_datetime(
            pattern_indicators_df['Date'])

        return pattern_indicators_df

## 3.0. Indices

In [19]:
# Define the tickers
indices_tickers = [
    "^GSPC",   # S&P 500
    "^FTSE",   # FTSE 100
    "^FCHI",   # CAC 40
    "^GDAXI",  # DAX Index (Germany)
    "^J203.JO" # JSE All Share Index (South Africa)
]

### to Merge

In [20]:
indices_growth_data = download_and_process_data(indices_tickers)
indices_growth_data_var = assign_growth_data_to_variables(indices_growth_data)
indices_growth_data_toMerge = merge_dataframes(indices_growth_data_var, on='Date')
# indices_growth_data_toMerge = merge_dataframes(indices_growth_data_var)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


## 4.0. Macro indicators

In [21]:
macro_indicators = {
    "GDP": {
        "SPDR S&P 500 ETF Trust": "SPY",
        "iShares MSCI ACWI ETF": "ACWI"
    },
    "Employment Data": {
        "SPDR S&P 500 ETF Trust": "SPY"
    },
    "Inflation Data": {
        "iShares TIPS Bond ETF": "TIP",
        "Schwab U.S. TIPS ETF": "SCHP"
    },
    "Interest Rates": {
        "iShares 20+ Year Treasury Bond ETF": "TLT",
        "Vanguard Short-Term Treasury ETF": "VGSH"
    }

}


### to Merge

In [22]:
macro_indicators_tickers = [ticker for sublist in macro_indicators.values() for ticker in sublist.values()]

macro_growth_data = download_and_process_data(macro_indicators_tickers)
macro_growth_data_var = assign_growth_data_to_variables(macro_growth_data)
macro_growth_data_toMerge = merge_dataframes(macro_growth_data_var, on='Date')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [23]:
macro_growth_data_toMerge.keys()

Index(['Date', 'growth_SPY_1d', 'growth_SPY_3d', 'growth_SPY_7d',
       'growth_SPY_30d', 'growth_SPY_90d', 'growth_SPY_365d', 'growth_ACWI_1d',
       'growth_ACWI_3d', 'growth_ACWI_7d', 'growth_ACWI_30d',
       'growth_ACWI_90d', 'growth_ACWI_365d', 'growth_TIP_1d', 'growth_TIP_3d',
       'growth_TIP_7d', 'growth_TIP_30d', 'growth_TIP_90d', 'growth_TIP_365d',
       'growth_SCHP_1d', 'growth_SCHP_3d', 'growth_SCHP_7d', 'growth_SCHP_30d',
       'growth_SCHP_90d', 'growth_SCHP_365d', 'growth_TLT_1d', 'growth_TLT_3d',
       'growth_TLT_7d', 'growth_TLT_30d', 'growth_TLT_90d', 'growth_TLT_365d',
       'growth_VGSH_1d', 'growth_VGSH_3d', 'growth_VGSH_7d', 'growth_VGSH_30d',
       'growth_VGSH_90d', 'growth_VGSH_365d'],
      dtype='object')

In [24]:
end = date.today()
print(f'Year = {end.year}; month= {end.month}; day={end.day}')

start = date(year=end.year-70, month=end.month, day=end.day)
print(f'Period for indexes: {start} to {end} ')


Year = 2024; month= 8; day=12
Period for indexes: 1954-08-12 to 2024-08-12 


In [25]:
# Real Potential Gross Domestic Product (GDPPOT), Billions of Chained 2012 Dollars, QUARTERLY
gdppot = pdr.DataReader("GDPPOT", "fred", start=start)
gdppot['gdppot_us_yoy'] = gdppot.GDPPOT/gdppot.GDPPOT.shift(4)-1
gdppot['gdppot_us_qoq'] = gdppot.GDPPOT/gdppot.GDPPOT.shift(1)-1
gdppot_to_merge = gdppot[['gdppot_us_yoy','gdppot_us_qoq']]

# # "Core CPI index", MONTHLY
cpilfesl = pdr.DataReader("CPILFESL", "fred", start=start)
cpilfesl['cpi_core_yoy'] = cpilfesl.CPILFESL/cpilfesl.CPILFESL.shift(12)-1
cpilfesl['cpi_core_mom'] = cpilfesl.CPILFESL/cpilfesl.CPILFESL.shift(1)-1
cpilfesl_to_merge = cpilfesl[['cpi_core_yoy','cpi_core_mom']]

# Fed rate https://fred.stlouisfed.org/series/FEDFUNDS
fedfunds = pdr.DataReader("FEDFUNDS", "fred", start=start)

# https://fred.stlouisfed.org/series/DGS1
dgs1 = pdr.DataReader("DGS1", "fred", start=start)
dgs5 = pdr.DataReader("DGS5", "fred", start=start)
dgs10 = pdr.DataReader("DGS10", "fred", start=start)





## 4.0. Volatility Indices


In [26]:


volatility_indices = {
    "CBOE Volatility Indices": {
        "CBOE Nasdaq-100 Volatility Index": "^VXN",
        "Volatility Index": "^VIX",
    },
    "Other Volatility Indices": {
        "CBOE 3-Month Volatility Index": "^VIX3M",
        "CBOE 6-Month Volatility Index": "^VIX6M"
    }
}



### to Merge

In [27]:
volatility_indicators_tickers = [ticker for sublist in volatility_indices.values() for ticker in sublist.values()]

volatility_growth_data = download_and_process_data(volatility_indicators_tickers)
volatility_growth_data_var = assign_growth_data_to_variables(volatility_growth_data)
volatility_growth_data_toMerge = merge_dataframes(volatility_growth_data_var, on='Date')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


## 5.0. Metals, Energy & Crypto assets

### 7.1. Metals

In [28]:
metal = {
    "Precious Metals": {
        "Gold": "GC=F",          # Gold Futures
        "Silver": "SI=F",        # Silver Futures
        "Platinum": "PL=F",      # Platinum Futures
        "Palladium": "PA=F"      # Palladium Futures
    }
}

In [29]:

metal_tickers = [ticker for sublist in metal.values() for ticker in sublist.values()]

metal_energy_growth_data = download_and_process_data(metal_tickers)
metal_growth_data_var = assign_growth_data_to_variables(metal_energy_growth_data)
metal_growth_data_toMerge = merge_dataframes(metal_growth_data_var, on='Date')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


### 7.2. Energy

In [30]:

energy = {
    "Energy Commodities": {
        "WTI Crude Oil": "CL=F", # WTI Crude Oil Futures
        "Brent Oil": "BZ=F",     # Brent Crude Oil Futures
        "Natural Gas": "NG=F",   # Natural Gas Futures
        "Heating Oil": "HO=F"    # Heating Oil Futures
    }
}


In [31]:

energy_tickers = [ticker for sublist in energy.values() for ticker in sublist.values()]

energy_growth_data = download_and_process_data(energy_tickers)
energy_growth_data_var = assign_growth_data_to_variables(energy_growth_data)
energy_growth_data_toMerge = merge_dataframes(energy_growth_data_var, on='Date')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


### 7.3. Crypto

In [32]:

crypto = {
    "Cryptocurrencies": {
        "Bitcoin": "BTC-USD",    # Bitcoin
        "Ethereum": "ETH-USD",   # Ethereum
        "Litecoin": "LTC-USD",   # Litecoin
        "Ripple": "XRP-USD"      # Ripple
    }
}


In [33]:

crypto_tickers = [ticker for sublist in crypto.values() for ticker in sublist.values()]

crypto_growth_data = download_and_process_data(crypto_tickers)
crypto_growth_data_var = assign_growth_data_to_variables(crypto_growth_data)
crypto_growth_data_toMerge = merge_dataframes(crypto_growth_data_var, on='Date')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


## 6.0. ETFs

In [34]:
# Define the ETF tickers
etf_tickers = {
    "Market Indices ETFs": {
        "SPDR S&P 500 ETF": "SPY",
        "Invesco QQQ Trust": "QQQ",
        "iShares Russell 2000 ETF": "IWM",
        "Vanguard Total Stock Market ETF": "VTI"
    },
    "International ETFs": {
        "iShares MSCI Emerging Markets ETF": "EEM"
    },
    "South African ETFs": {
        "Satrix 40 ETF": "STX40.JO",
        "Satrix RESI ETF": "STXRES.JO",
        "Satrix FINI ETF": "STXFIN.JO",
        "Satrix INDI ETF": "STXIND.JO",
    }
}



### Merge

In [35]:

ExtFormatter_tickers = [ticker for sublist in etf_tickers.values() for ticker in sublist.values()]

etf_growth_data = download_and_process_data(ExtFormatter_tickers)
etf_growth_data_var = assign_growth_data_to_variables(etf_growth_data)
etf_growth_data_toMerge = merge_dataframes(etf_growth_data_var, on='Date')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


### 7.0. Currencies

#### Forex Currency Pairs (Daily Data)

In [36]:
# forex_currency_pairs_daily = {
#     'Euro vs US Dollar': 'EURUSD=X',
#     'US Dollar vs Japanese Yen': 'JPY=X',
#     'British Pound vs US Dollar': 'GBPUSD=X',
#     'US Dollar vs Swiss Franc': 'CHF=X',
#     'Australian Dollar vs US Dollar': 'AUDUSD=X',
#     'US Dollar vs Canadian Dollar': 'CAD=X',
#     'New Zealand Dollar vs US Dollar': 'NZDUSD=X',
#     'Euro vs British Pound': 'EURGBP=X',
#     'Euro vs Japanese Yen': 'EURJPY=X',
#     'British Pound vs Japanese Yen': 'GBPJPY=X',
#     'Australian Dollar vs Japanese Yen': 'AUDJPY=X',
# }


# african_currency_pairs_daily = {
#     'US Dollar vs South African Rand': 'USDZAR=X',
#     'Euro vs South African Rand': 'EURZAR=X',
#     'British Pound vs South African Rand': 'GBPZAR=X',
#     'US Dollar vs Nigerian Naira': 'USDNGN=X',
#     'US Dollar vs Kenyan Shilling': 'USDKES=X',
#     'US Dollar vs Ghanaian Cedi': 'USDGHS=X',
#     'US Dollar vs Egyptian Pound': 'USDEGP=X'
# }



In [39]:
african_currency_pairs_daily = {
    'US Dollar vs South African Rand': 'USDZAR=X',
    'Euro vs South African Rand': 'EURZAR=X',
    'British Pound vs South African Rand': 'GBPZAR=X',
    'US Dollar vs Nigerian Naira': 'USDNGN=X',
    'US Dollar vs Kenyan Shilling': 'USDKES=X',
    'US Dollar vs Ghanaian Cedi': 'USDGHS=X',
    'US Dollar vs Egyptian Pound': 'USDEGP=X'
}
usa_currency_pairs_daily = {
    'US Dollar vs Euro': 'EURUSD=X',
    'US Dollar vs Japanese Yen': 'USDJPY=X',
    'US Dollar vs British Pound': 'GBPUSD=X',
    'US Dollar vs Swiss Franc': 'USDCHF=X',
    'US Dollar vs Canadian Dollar': 'USDCAD=X',
    'US Dollar vs Australian Dollar': 'AUDUSD=X',
    'US Dollar vs Chinese Yuan': 'USDCNY=X'
}
south_america_currency_pairs_daily = {
    'US Dollar vs Brazilian Real': 'USDBRL=X',
    'US Dollar vs Argentine Peso': 'USDARS=X',
    'US Dollar vs Chilean Peso': 'USDCLP=X',
    'US Dollar vs Colombian Peso': 'USDCOP=X',
    'US Dollar vs Peruvian Sol': 'USDPEN=X',
    # 'US Dollar vs Paraguayan Guarani': 'USDOPY=X',
    'US Dollar vs Uruguayan Peso': 'USDUYU=X'
}
europe_currency_pairs_daily = {
    'Euro vs British Pound': 'EURGBP=X',
    'Euro vs Swiss Franc': 'EURCHF=X',
    'Euro vs Japanese Yen': 'EURJPY=X',
    'Euro vs Norwegian Krone': 'EURNOK=X',
    'Euro vs Swedish Krona': 'EURSEK=X',
    'Euro vs Danish Krone': 'EURDKK=X',
    'British Pound vs Swiss Franc': 'GBPCHF=X'
}
asia_currency_pairs_daily = {
    'US Dollar vs Japanese Yen': 'USDJPY=X',
    'US Dollar vs Chinese Yuan': 'USDCNY=X',
    'US Dollar vs Indian Rupee': 'USDINR=X',
    'US Dollar vs South Korean Won': 'USDKRW=X',
    'US Dollar vs Singapore Dollar': 'USDSGD=X',
    'US Dollar vs Hong Kong Dollar': 'USDHKD=X',
    'US Dollar vs Thai Baht': 'USDTHB=X'
}


In [40]:
def combine_tickers(*dicts):
    combined_dict = {}
    for d in dicts:
        combined_dict.update(d)
    return list(combined_dict.values())

# Define the dictionaries (assuming they are already defined as south_africa_tickers, usa_tickers, eu_tickers, india_tickers, china_tickers)

# ALL_TICKERS = combine_tickers(forex_currency_pairs_daily, african_currency_pairs_daily)
# currency_df = process_all_tickers(ALL_TICKERS)


ALL_TICKERS = combine_tickers(
    african_currency_pairs_daily,
    usa_currency_pairs_daily,
    south_america_currency_pairs_daily,
    europe_currency_pairs_daily,
    asia_currency_pairs_daily
    )
currency_df = process_all_tickers(ALL_TICKERS)


[*********************100%%**********************]  1 of 1 completed

0 USDZAR=X



[*********************100%%**********************]  1 of 1 completed

1 EURZAR=X



[*********************100%%**********************]  1 of 1 completed

...

[*********************100%%**********************]  1 of 1 completed


30 USDHKD=X


[*********************100%%**********************]  1 of 1 completed


31 USDTHB=X


[*********************100%%**********************]  1 of 1 completed


In [41]:
ALL_TICKERS

['USDZAR=X',
 'EURZAR=X',
 'GBPZAR=X',
 'USDNGN=X',
 'USDKES=X',
 'USDGHS=X',
 'USDEGP=X',
 'EURUSD=X',
 'USDJPY=X',
 'GBPUSD=X',
 'USDCHF=X',
 'USDCAD=X',
 'AUDUSD=X',
 'USDCNY=X',
 'USDBRL=X',
 'USDARS=X',
 'USDCLP=X',
 'USDCOP=X',
 'USDPEN=X',
 'USDUYU=X',
 'EURGBP=X',
 'EURCHF=X',
 'EURJPY=X',
 'EURNOK=X',
 'EURSEK=X',
 'EURDKK=X',
 'GBPCHF=X',
 'USDINR=X',
 'USDKRW=X',
 'USDSGD=X',
 'USDHKD=X',
 'USDTHB=X']

In [42]:
currency_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179061 entries, 0 to 179060
Data columns (total 24 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Open                          179061 non-null  float64
 1   High                          179061 non-null  float64
 2   Low                           179061 non-null  float64
 3   Close                         179061 non-null  float64
 4   Adj Close                     179061 non-null  float64
 5   Volume                        179061 non-null  int64  
 6   Ticker                        179061 non-null  object 
 7   Year                          179061 non-null  int32  
 8   Month                         179061 non-null  int32  
 9   Weekday                       179061 non-null  int32  
 10  Date                          179061 non-null  object 
 11  growth_1d                     179029 non-null  float64
 12  growth_3d                     178965 non-nul

In [43]:
currency_df.tail()

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_30d,growth_90d,growth_365d,growth_future_5d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_5d_future
179056,35.500000,35.590000,35.349998,35.500000,35.500000,0,USDTHB=X,2024,8,1,...,0.965724,0.975194,1.026457,NaN,35.803900,35.98255,0,0.006761,6.783520,0
179057,35.393002,35.709999,35.402000,35.393002,35.393002,0,USDTHB=X,2024,8,2,...,0.967075,0.967286,1.025290,NaN,35.729200,35.93170,0,0.008702,7.084752,0
179058,35.580002,35.592999,35.196999,35.580002,35.580002,0,USDTHB=X,2024,8,3,...,0.968691,0.972876,1.030050,NaN,35.677401,35.89470,0,0.011130,7.085785,0
179059,35.259998,35.299999,35.147999,35.259998,35.259998,0,USDTHB=X,2024,8,4,...,0.954702,0.963862,1.020255,NaN,35.582401,35.85625,0,0.004311,7.110741,0
179060,35.180000,35.271999,35.150002,35.220001,35.220001,0,USDTHB=X,2024,8,0,...,0.956545,0.960249,1.022945,NaN,35.507401,35.80825,0,0.003464,7.177170,0


In [44]:
# currency_df['ticker_type'] = currency_df.Ticker.apply(
#     lambda x:get_ticker_type(
#         x,
#         list(forex_currency_pairs_daily.values()),
#         list(african_currency_pairs_daily.values()),
#         )
#     )


In [47]:
currency_df['ticker_type'] = currency_df.Ticker.apply(
    lambda x:get_ticker_type(
        x,
        list(african_currency_pairs_daily.values()),
        list(usa_currency_pairs_daily.values()),
        list(south_america_currency_pairs_daily.values()),
        list(europe_currency_pairs_daily.values()),
        list(asia_currency_pairs_daily.values()),
        )
    )


In [48]:
currency_df.ticker_type.value_counts()


,count
ticker_type,
europe,41682
usa,39355
africa,38116
south_america,32589
asia,27319


In [49]:
# Volume needs to be float, not int
currency_df['Volume'] = currency_df['Volume']*1.0

In [50]:
# to resolve an error "Exception: input array type is not double"
# https://stackoverflow.com/questions/51712269/how-to-run-ta-lib-on-multiple-columns-of-a-pandas-dataframe
for f in ['Open','High','Low','Close', 'Volume', 'Adj Close']:
  currency_df.loc[:,f] = currency_df.loc[:,f].astype('float64')

In [51]:
# supress warnings
pd.options.mode.chained_assignment = None  # default='warn'

# adding Momentum / Pattern/ Volume features to all tickers - one by one
merged_df_with_tech_ind = pd.DataFrame({'A' : []})

current_ticker_data = None
i=0

for ticker in ALL_TICKERS:
  i+=1
  print(f'{i}/{len(ALL_TICKERS)} Current ticker is {ticker}')
  current_ticker_data = currency_df[currency_df.Ticker.isin([ticker])]
  # need to have same 'utc' time on both sides
  # https://stackoverflow.com/questions/73964894/you-are-trying-to-merge-on-datetime64ns-utc-and-datetime64ns-columns-if-yo
  current_ticker_data['Date']= pd.to_datetime(current_ticker_data['Date'], utc=True)

  # 3 calls to get additional features
  df_current_ticker_momentum_indicators = talib_get_momentum_indicators_for_one_ticker(current_ticker_data)
  df_current_ticker_momentum_indicators["Date"]= pd.to_datetime(df_current_ticker_momentum_indicators['Date'], utc=True)
  # df_current_ticker_momentum_indicators.loc[:,"Date"]= pd.to_datetime(df_current_ticker_momentum_indicators['Date'], utc=True)

  df_current_ticker_volume_indicators = talib_get_volume_volatility_cycle_price_indicators(current_ticker_data)
  df_current_ticker_volume_indicators["Date"]= pd.to_datetime(df_current_ticker_volume_indicators['Date'], utc=True)
  # df_current_ticker_volume_indicators.loc[:,"Date"]= pd.to_datetime(df_current_ticker_volume_indicators['Date'], utc=True)

  df_current_ticker_pattern_indicators = talib_get_pattern_recognition_indicators(current_ticker_data)
  df_current_ticker_pattern_indicators["Date"]= pd.to_datetime(df_current_ticker_pattern_indicators['Date'], utc=True)
  # df_current_ticker_pattern_indicators.loc[:,"Date"]= pd.to_datetime(df_current_ticker_pattern_indicators['Date'], utc=True)

  # merge to one df
  m1 = pd.merge(current_ticker_data, df_current_ticker_momentum_indicators.reset_index(), how = 'left', on = ["Date","Ticker"], validate = "one_to_one")
  m2 = pd.merge(m1, df_current_ticker_volume_indicators.reset_index(), how = 'left', on = ["Date","Ticker"], validate = "one_to_one")
  m3 = pd.merge(m2, df_current_ticker_pattern_indicators.reset_index(), how = 'left', on = ["Date","Ticker"], validate = "one_to_one")

  if merged_df_with_tech_ind.empty:
    merged_df_with_tech_ind = m3
  else:
    merged_df_with_tech_ind = pd.concat([merged_df_with_tech_ind,m3], ignore_index = False)

1/32 Current ticker is USDZAR=X
2/32 Current ticker is EURZAR=X
3/32 Current ticker is GBPZAR=X
4/32 Current ticker is USDNGN=X
5/32 Current ticker is USDKES=X
6/32 Current ticker is USDGHS=X
7/32 Current ticker is USDEGP=X
8/32 Current ticker is EURUSD=X
9/32 Current ticker is USDJPY=X
10/32 Current ticker is GBPUSD=X
11/32 Current ticker is USDCHF=X
12/32 Current ticker is USDCAD=X
13/32 Current ticker is AUDUSD=X
14/32 Current ticker is USDCNY=X
15/32 Current ticker is USDBRL=X
16/32 Current ticker is USDARS=X
17/32 Current ticker is USDCLP=X
18/32 Current ticker is USDCOP=X
19/32 Current ticker is USDPEN=X
20/32 Current ticker is USDUYU=X
21/32 Current ticker is EURGBP=X
22/32 Current ticker is EURCHF=X
23/32 Current ticker is EURJPY=X
24/32 Current ticker is EURNOK=X
25/32 Current ticker is EURSEK=X
26/32 Current ticker is EURDKK=X
27/32 Current ticker is GBPCHF=X
28/32 Current ticker is USDINR=X
29/32 Current ticker is USDKRW=X
30/32 Current ticker is USDSGD=X
31/32 Current ticke

In [52]:
merged_df_with_tech_ind['Date'] = pd.to_datetime(merged_df_with_tech_ind['Date']).dt.tz_localize(None)
merged_df_with_tech_ind.tail()

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,cdlspinningtop,cdlstalledpattern,cdlsticksandwich,cdltakuru,cdltasukigap,cdlthrusting,cdltristar,cdlunique3river,cdlupsidegap2crows,cdlxsidegap3methods
5365,35.500000,35.590000,35.349998,35.500000,35.500000,0.0,USDTHB=X,2024,8,1,...,100,0,0,0,0,0,0,0,0,0
5366,35.393002,35.709999,35.402000,35.393002,35.393002,0.0,USDTHB=X,2024,8,2,...,0,0,0,0,0,0,0,0,0,0
5367,35.580002,35.592999,35.196999,35.580002,35.580002,0.0,USDTHB=X,2024,8,3,...,100,0,0,100,0,0,100,0,0,0
5368,35.259998,35.299999,35.147999,35.259998,35.259998,0.0,USDTHB=X,2024,8,4,...,100,0,0,0,0,0,-100,0,0,0
5369,35.180000,35.271999,35.150002,35.220001,35.220001,0.0,USDTHB=X,2024,8,0,...,0,0,0,0,0,0,0,0,0,0


## 9.0. Merge

In [53]:
def ensure_datetime(df):
    """
    Ensure the 'Date' column is in datetime format without timezone.

    Parameters:
    df (pd.DataFrame): DataFrame to process.

    Returns:
    pd.DataFrame: DataFrame with 'Date' column in datetime format.
    """
    df['Date'] = pd.to_datetime(df['Date']).dt.tz_localize(None)
    return df

def remove_duplicate_columns(df):
    """
    Remove duplicate columns resulting from merge operations.

    Parameters:
    df (pd.DataFrame): DataFrame to process.

    Returns:
    pd.DataFrame: DataFrame with duplicate columns removed.
    """
    cols_to_drop = [col for col in df.columns if '_dup' in col]
    df = df.drop(columns=cols_to_drop)
    return df

def final_merge_dataframes(dataframe_names, on='Date', how='left') -> pd.DataFrame:
    """
    Merge multiple DataFrames on a specified column.

    Parameters:
    dataframe_names (list of str): List of DataFrame names to merge.
    on (str): The column name to merge on.
    how (str): The type of merge to be performed (default is 'left').

    Returns:
    pd.DataFrame: The merged DataFrame.
    """
    if not dataframe_names:
        raise ValueError("The list of DataFrame names is empty.")

    # Initialize with the first DataFrame
    merged_df = ensure_datetime(globals()[dataframe_names[0]])

    for name in dataframe_names[1:]:
        df = ensure_datetime(globals()[name])
        merged_df = pd.merge(merged_df, df, on=on, how=how, suffixes=('', '_dup'))
        merged_df = remove_duplicate_columns(merged_df)

    return merged_df


In [54]:

batch_1 = [
    'merged_df_with_tech_ind',
    'indices_growth_data_toMerge',
    'macro_growth_data_toMerge',
    'crypto_growth_data_toMerge',
    'etf_growth_data_toMerge',
    'volatility_growth_data_toMerge',
    'metal_growth_data_toMerge',
    'energy_growth_data_toMerge',
    # 'gdppot_to_merge',
    # 'cpilfesl_to_merge',
    # 'fedfunds',
    # 'dgs1',
    # 'dgs5',
    # 'dgs10',
]

final_merged_df = final_merge_dataframes(batch_1)


In [55]:
final_merged_df['Date'] = pd.to_datetime(final_merged_df['Date']).dt.tz_localize(None)


In [56]:
final_merged_df['Quarter'] = final_merged_df['Date'].dt.to_period('Q').dt.to_timestamp()


final_merged_df_m6 = pd.merge(final_merged_df,
        gdppot_to_merge,
        how='left',
        left_on='Quarter',
        right_index=True,
        validate = "many_to_one"
          )

In [57]:
final_merged_df_m6['Month'] = final_merged_df_m6['Date'].dt.to_period('M').dt.to_timestamp()
final_merged_df_m7 = pd.merge(final_merged_df_m6,
              cpilfesl_to_merge,
              how='left',
              left_on='Month',
              right_index=True,
              validate = "many_to_one"
              )

In [58]:

fields_to_fill = ['cpi_core_yoy',	'cpi_core_mom']
# Fill missing values in selected fields with the last defined value
for field in fields_to_fill:
    final_merged_df_m7[field] = final_merged_df_m7[field].fillna(method='ffill')

<ipython-input-58-012a844f0b78>:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  final_merged_df_m7[field] = final_merged_df_m7[field].fillna(method='ffill')


In [59]:

final_merged_df_m8 = pd.merge(
    final_merged_df_m7,
    fedfunds,
    how='left',
    left_on='Month',
    right_index=True,
    validate = "many_to_one"
    )

In [60]:

fields_to_fill = ['FEDFUNDS']
# Fill missing values in selected fields with the last defined value
for field in fields_to_fill:
    final_merged_df_m8[field] = final_merged_df_m8[field].fillna(method='ffill')


<ipython-input-60-c9df2ee221f1>:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  final_merged_df_m8[field] = final_merged_df_m8[field].fillna(method='ffill')


In [61]:

final_merged_df_m9 = pd.merge(
    final_merged_df_m8,
    dgs1,
    how='left',
    left_on='Date',
    right_index=True,
    validate = "many_to_one"
    )


final_merged_df_m10 = pd.merge(
    final_merged_df_m9,
    dgs5,
    how='left',
    left_on='Date',
    right_index=True,
    validate = "many_to_one"
    )


final_merged_df_m11 = pd.merge(
    final_merged_df_m10,
    dgs10,
    how='left',
    left_on='Date',
    right_index=True,
    validate = "many_to_one"
    )

In [62]:
final_merged_df_m11.tail().T

,179056,179057,179058,179059,179060
Open,35.5,35.393002,35.580002,35.259998,35.18
High,35.59,35.709999,35.592999,35.299999,35.271999
Low,35.349998,35.402,35.196999,35.147999,35.150002
Close,35.5,35.393002,35.580002,35.259998,35.220001
Adj Close,35.5,35.393002,35.580002,35.259998,35.220001
...,...,...,...,...,...
cpi_core_mom,0.000648,0.000648,0.000648,0.000648,0.000648
FEDFUNDS,5.33,5.33,5.33,5.33,5.33
DGS1,4.46,4.45,4.48,NaN,NaN
DGS5,3.73,3.79,3.83,NaN,NaN


# to Disk

In [63]:

date = final_merged_df_m11.Date.max()
date_str = date.strftime('%Y_%m_%d')

final_merged_df_m11.to_parquet(f'forexdf_{date_str}.parquet.brotli',
              compression='brotli')